In [1]:
using DataFrames, CSV, DelimitedFiles, JuMP, Gurobi
const GRB_ENV = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x0000000067d32d20, false, 0)

In [2]:
NUM_CREWS = 10                
BREAK_LENGTH = 2       # how long at base to be considered "rested"

# tradeoffs
BETA = 100             # cost of one area unit burned / cost of mile traveled
ALPHA = 200            # cost of crew-day of suppression / cost of mile traveled
LINE_PER_CREW = 17     # how much perimeter prevented per crew per time period

FIRE_CODE = 1
BASE_CODE = 2

2

In [3]:
struct GlobalData
    
    ff_dist::Matrix{Float64}
    bf_dist::Matrix{Float64}
    ff_tau::Matrix{Int64}
    bf_tau::Matrix{Int64}
    
end

struct CrewStatus
    
    rest_by::Vector{Int64}
    current_fire::Vector{Int64}
    rested_periods::Vector{Int64}
    
end

struct RegionData
    
    crew_regions::Vector{Int64}
    fire_regions::Vector{Int64}
    
end


struct KeyArcIndices
    
    # fire flow data
    f_out::Array{Vector{Int64}}
    f_in::Array{Vector{Int64}}
    
    # base flow data
    b_out::Array{Vector{Int64}}
    b_in::Array{Vector{Int64}}
    
    # total crews suppressing each fire
    supp_fire::Array{Vector{Int64}}
    
    # start constraints
    start::Array{Vector{Int64}}
    
    # assignments out of region
    out_of_region::Array{Vector{Int64}}
    
end 

mutable struct RouteData
    
    routes_per_crew::Vector{Int64} # could add in length
    route_costs::Matrix{Float64}
    fires_fought::BitArray{4}
    out_of_reg::BitArray{3}
    
end

mutable struct SuppressionPlanData
    
    plans_per_fire::Vector{Int64} # could add in length
    plan_costs::Matrix{Float64}
    crews_present::Array{Int8, 3}
    
end



In [4]:
struct FireConstraintIndices
    
    # fire flow data
    flow_out::Array{Vector{Int64}}
    flow_in::Array{Vector{Int64}}
    
    # total crews suppressing each fire
    crews_needed::Array{Vector{Int64}}
    
    # start constraints
    start::Array{Vector{Int64}}
    
end

In [5]:
struct ColumnGeneration
    
    route_sps::Vector{Any}
    plan_sps::Vector{Any}
    routes::RouteData
    suppression_plans::SuppressionPlanData
    
end

In [6]:
function get_rotation_orders(crew_regions)
    
    # initialize output
    out = Dict()
    
    # get the unique regions where there are crews
    regions = unique(crew_regions)
    
    # for each region
    for region in regions
        
        # initialize dictionary corresponding to the order
        out[region] = Dict() 
        crews_in_region = 0
        
        # for each crew in the region
        for crew in 1:NUM_CREWS
            
            if crew_regions[crew] == region
                
                # update crew count, log rotation order 
                crews_in_region += 1
                out[region][crew] = crews_in_region
            end
        end
    end
    
    return out
end

get_rotation_orders (generic function with 1 method)

In [7]:
# crew, from_type, from_ix, to_type, to_ix, from_time, to_time, from_rested, to_rested, exited_region

In [8]:
function arc_exits_region(crew, from_type, from_ix, to_type, to_ix, region_data)
    
    # get the region where the arc originates
    from_region = 0
    if from_type == FIRE_CODE
        from_region = region_data.fire_regions[from_ix]
    elseif from_type == BASE_CODE
        from_region = region_data.crew_regions[from_ix]
    else
        throw(DomainError(from_type, "from_type invalid"))
    end
    
    # get the region where the arc terminates
    to_region = 0
    if to_type == FIRE_CODE
        to_region = region_data.fire_regions[to_ix]
    elseif to_type == BASE_CODE
        to_region = region_data.crew_regions[to_ix]
    else
        throw(DomainError(from_type, "to_type invalid"))
    end
    
    # if these are different regions
    if from_region != to_region
        
        # if the crew is leaving its home region
        if region_data.crew_regions[crew] == from_region
        
            # return the region that the arc exited
            return from_region
        
        end
        
    end
    
    # otherwise
    return 0
    
end     

arc_exits_region (generic function with 1 method)

In [9]:
function generate_arcs(gd, rd, cs)
    
    # get fire-to-fire arcs
    ff = [[c, FIRE_CODE, f_from, FIRE_CODE, f_to, t_from, t_from + gd.ff_tau[f_to, f_from], rest, rest]
          for c=1:NUM_CREWS, f_from=1:NUM_FIRES, f_to=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    ff = copy(reduce(hcat, ff)')

    # get fire-to-fire arcs from start, based on cs.current crew locations
    from_start_ff = [[c, FIRE_CODE, cs.current_fire[c], FIRE_CODE, f_to, 0, gd.ff_tau[f_to, cs.current_fire[c]], 0, 0]
                      for c=1:NUM_CREWS, f_to=1:NUM_FIRES if cs.current_fire[c] != -1]
    from_start_ff = copy(reduce(hcat, from_start_ff)')

    # get base-to-fire arcs
    rf = [[c, BASE_CODE, c, FIRE_CODE, f_to, t_from, t_from + gd.bf_tau[c, f_to], rest, rest]
           for c=1:NUM_CREWS, f_to=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    rf = copy(reduce(hcat, rf)')

    # get base-to-fire arcs from start
    from_start_rf = [[c, BASE_CODE, c, FIRE_CODE, f_to, 0, gd.bf_tau[c, f_to], 0, 0]
                      for c=1:NUM_CREWS, f_to=1:NUM_FIRES if cs.current_fire[c] == -1]
    from_start_rf = copy(reduce(hcat, from_start_rf)')

    # get fire-to-base arcs
    fr = [[c, FIRE_CODE, f_from, BASE_CODE, c, t_from, t_from + gd.bf_tau[c, f_from], rest, rest]
           for c=1:NUM_CREWS, f_from=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    fr = copy(reduce(hcat, fr)')

    # get fire-to-base arcs from start, based on cs.current crew locations
    from_start_fr = [[c, FIRE_CODE, cs.current_fire[c], BASE_CODE, c, 0, gd.bf_tau[c, cs.current_fire[c]], 0, 0]
                      for c=1:NUM_CREWS if cs.current_fire[c] != -1]
    from_start_fr = copy(reduce(hcat, from_start_fr)')

    # get base-to-base arcs
    rr = [[c, BASE_CODE, c, BASE_CODE, c, t_from, t_from + 1 + (BREAK_LENGTH - 1) * rest, 0, rest]
          for c=1:NUM_CREWS, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    rr = copy(reduce(hcat, rr)')
    rr_rested = [[c, BASE_CODE, c, BASE_CODE, c, t_from, t_from + 1, 1, 1]
          for c=1:NUM_CREWS, t_from=1:NUM_TIME_PERIODS]
    rr_rested  = copy(reduce(hcat, rr_rested)')

    # get base-to-base arcs from start, based on cs.current days rested
    from_start_rr = [[c, BASE_CODE, c, BASE_CODE, c, 0, 
                      1 + (BREAK_LENGTH - max(cs.rested_periods[c], 0) - 1) * rest, 0, rest] 
                      for c=1:NUM_CREWS, rest=0:1 if cs.current_fire[c] == -1]
    from_start_rr = copy(reduce(hcat, from_start_rr)')

    A = vcat(ff, from_start_ff, rf, from_start_rf, fr, from_start_fr, rr, rr_rested, from_start_rr)

    out_of_region = [arc_exits_region(A[i, 1], A[i, 2], A[i, 3], A[i, 4], A[i, 5], rd) 
                     for i in 1:length(A[:, 1])]
    A = hcat(A, out_of_region)
    
    return A
end

generate_arcs (generic function with 1 method)

In [10]:
function get_distance(from_type, from_ix, to_type, to_ix, fire_fire, base_fire)
    
    dist = 0
    
    # if fire to fire
    if (from_type == FIRE_CODE) & (to_type == FIRE_CODE)
        dist = fire_fire[from_ix, to_ix]
    
    # if fire to base
    elseif (from_type == FIRE_CODE) & (to_type == BASE_CODE)
        dist = base_fire[to_ix, from_ix]
    
    # if base to fire
    elseif (from_type == BASE_CODE) & (to_type == FIRE_CODE)
        dist = base_fire[from_ix, to_ix]
        
    # otherwise dist still 0
    end
    
    return dist
end 

get_distance (generic function with 1 method)

In [11]:
function get_arc_costs(gd, arcs, cost_param_dict)
    
    # get number of arcs
    n_arcs = length(arcs[:, 1])
    
    # initialize costs to 0
    costs = zeros(n_arcs)
    
    # if there is travel cost per mile
    if "cost_per_mile" in keys(cost_param_dict)
        
        # find the miles for each arc
        miles_per_arc =  [get_distance(arcs[i, 2], arcs[i, 3], 
                                       arcs[i, 4], arcs[i, 5], 
                                       gd.ff_dist, gd.bf_dist) for i in 1:n_arcs]
        # add to costs
        costs = costs .+ (cost_param_dict["cost_per_mile"] * miles_per_arc)
    end
    
    # if there are rest violations
    if "rest_violation" in keys(cost_param_dict)
        
        # find the rest violation scores
        rest_violation_matrix = cost_param_dict["rest_violation"]
        rest_violations = [(arcs[i, 8] == 0) & (arcs[i, 6] > 0) ? 
                           rest_violation_matrix[arcs[i, 1], arcs[i, 6]] : 0
                           for i in 1:n_arcs]
        
        # add to costs
        costs = costs .+ rest_violations
    end
    
    if "fight_fire" in keys(cost_param_dict)
        costs = costs .+ [(arcs[i, 4] == FIRE_CODE) ? cost_param_dict["fight_fire"] : 0
                          for i in 1:n_arcs]
    end
    
    # if we have to adjust for linking dual constraints
    if "linking_dual" in keys(cost_param_dict)
        
        # get the dual variables
        rho = cost_param_dict["linking_dual"]
        
        # get linking costs (really benefits) if arc goes to a fire
        linking_costs = [((arcs[i, 4] == FIRE_CODE) & (arcs[i, 7] <= NUM_TIME_PERIODS)) ? 
                          - rho[arcs[i, 5], arcs[i, 7]] : 0
                          for i in 1:n_arcs]
        
        # add to costs
        costs = costs .+ linking_costs
        
    end
    
    # if we have to adjust for linking dual constraints
    if "out_of_region_dual" in keys(cost_param_dict)
        
        # get needed regional info
        regs = cost_param_dict["region_data"].crew_regions
        rot_order = cost_param_dict["rotation_order"]
        
        # get the dual variables
        eta = cost_param_dict["out_of_region_dual"]

        # get adjustment for crew allotment
        c1 = [(arcs[i, 10] > 0) ? sum(eta[arcs[i, 1], t_0]
                                        for t_0=arcs[i, 6]:NUM_TIME_PERIODS
                                      ) : 0
                                                   
               for i in 1:n_arcs
             ]
        
        # get adjustment for region average allotment
        c2 = [(arcs[i, 10] > 0) ? sum(eta[c, t_0]
                                            for c in keys(rot_order[regs[arcs[i, 1]]]),
                                                t_0=arcs[i, 6]:NUM_TIME_PERIODS) /
                                        length(keys(rot_order[regs[arcs[i, 1]]])) : 0
                                                   
               for i in 1:n_arcs
             ]
        
        # get adjustment for big-M constraint
        c3 = [(arcs[i, 10] > 0) ? NUM_TIME_PERIODS * eta[arcs[i, 1], arcs[i, 6]] : 0
               for i in 1:n_arcs
             ]
            
        # add to costs
        costs = costs .+ c1 .- c2 .+ c3
        
    end   
    
    return costs
end

get_arc_costs (generic function with 1 method)

In [12]:
function positive(x)
    
    if x > 0
        return 1
    end
    
    return 0
end

function is_one(x)
    
    if x == 1
        return 1
    end
    
    return 0
end

is_one (generic function with 1 method)

In [13]:
# should return matrix indexed by crew, time, 
function get_rest_penalties(rest_by_periods, lambda, accounting_func)
    
    penalties = zeros(NUM_CREWS, NUM_TIME_PERIODS)
    
    for c in 1:NUM_CREWS
        penalties[c, :] = [lambda * accounting_func(t - rest_by_periods[c]) 
                           for t in 1:NUM_TIME_PERIODS]
    end
    
    return penalties    
end

get_rest_penalties (generic function with 1 method)

In [14]:
function define_network_constraint_data(arcs)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # get number of arcs
    n_arcs = length(arcs[:, 1])
      
    ## flow balance ##
    
    # initialize arrays of vectors for flow balance
    f_out = Array{Vector{Int64}}(undef, C, G, T, 2)
    f_in = Array{Vector{Int64}}(undef, C, G, T, 2)
    b_out = Array{Vector{Int64}}(undef, C, T, 2)
    b_in = Array{Vector{Int64}}(undef, C, T, 2)
    start = Array{Vector{Int64}}(undef, C)
    out_of_region = Array{Vector{Int64}}(undef, C, T+1)
    
    # for each crew
    for crew in 1:C
        
        # get indices of this crew's arcs only
        crew_ixs = [i for i in 1:n_arcs if arcs[i, 1] == crew]
        
        # get time 0 indices
        start[crew] = [i for i in crew_ixs if arcs[i, 6] == 0]
        
        # for each time period (including start)
        for tm in 0:T
        
            # get indices for out of region assignments
            out_of_region[crew, tm+1] = [i for i in crew_ixs if
                                           (arcs[i, 6] == tm) &
                                           (arcs[i, 10] > 0)
                                        ]
        end
        
        # for each time period
        for tm in 1:T
            
            # for each rest state
            for rest in 1:2
                
                # get arcs leaving crew base at this time with this rest
                b_out[crew, tm, rest] = [i for i in crew_ixs if
                                         (arcs[i, 2] == BASE_CODE) &
                                         (arcs[i, 6] == tm) &
                                         (arcs[i, 8] == rest-1)
                                        ]
                
                # get arcs entering crew base at this time with this rest
                b_in[crew, tm, rest] = [i for i in crew_ixs if
                                        (arcs[i, 4] == BASE_CODE) &
                                        (arcs[i, 7] == tm) &
                                        (arcs[i, 9] == rest-1)
                                       ]
                # for each fire
                for fire in 1:G
                    
                    # get arcs where this crew leaves this fire at this time
                    # with this rest state
                    f_out[crew, fire, tm, rest] = [i for i in crew_ixs if
                                                   (arcs[i, 2] == FIRE_CODE) &
                                                   (arcs[i, 3] == fire) &
                                                   (arcs[i, 6] == tm) &
                                                   (arcs[i, 8] == rest-1)
                                                   ]
                    
                    # get arcs where this crew enters this fire at this time
                    # with this rest state
                    f_in[crew, fire, tm, rest] = [i for i in crew_ixs if
                                                  (arcs[i, 4] == FIRE_CODE) &
                                                  (arcs[i, 5] == fire) &
                                                  (arcs[i, 7] == tm) &
                                                  (arcs[i, 9] == rest-1)
                                                  ]
                end
            end
        end
    end
    
    ## linking constraints ##
    linking = Array{Vector{Int64}}(undef, G, T)
    for fire in 1:G
        for tm in 1:T
            
            # we count the crew as working *where they arrived* during this timestep
            linking[fire, tm] = [i for i in 1:n_arcs if (arcs[i, 4] == FIRE_CODE) &
                                                        (arcs[i, 5] == fire) &
                                                        (arcs[i, 7] == tm)]
        end
    end
    
    
    return KeyArcIndices(f_out, f_in, b_out, b_in, linking, start, out_of_region)
end

define_network_constraint_data (generic function with 1 method)

In [15]:
function get_route_stats(arc_ixs_used, arcs, costs)
    
    # get total cost
    route_cost = sum(costs[arc_ixs_used])
    
    # initialize fires fought matrix
    fires_fought =  falses(NUM_FIRES, NUM_TIME_PERIODS)
    
    # initialize out of region matrix
    out_of_region = falses(NUM_TIME_PERIODS + 1)
    
    # for each arc used
    for ix in arc_ixs_used
        arc = arcs[ix, :]
        
        # update fires_fought
        if (arc[4] == FIRE_CODE) & (arc[7] <= NUM_TIME_PERIODS)
            @assert ~fires_fought[arc[5], arc[7]] "Visited fire twice at same time"
            fires_fought[arc[5], arc[7]] = true
        end
        
        # update out_of_region
        if arc[10] > 0
            @assert ~out_of_region[arc[6] + 1] "Left region twice at same time"
            out_of_region[arc[6] + 1] = true
        end
    end
    
    return route_cost, fires_fought, out_of_region
end

get_route_stats (generic function with 1 method)

In [16]:
function initialize_route_data(max_routes)
    
    return RouteData(zeros(NUM_CREWS), Matrix{Float64}(undef, NUM_CREWS, max_routes),
                     BitArray(undef, NUM_CREWS, max_routes, NUM_FIRES, NUM_TIME_PERIODS) .> 2,
                     BitArray(undef, NUM_CREWS, max_routes, NUM_TIME_PERIODS + 1) .> 2)
end

initialize_route_data (generic function with 1 method)

In [17]:
function update_available_routes(crew, route_ixs, arcs, costs, route_data)
    
    # get the required information from the arcs used
    route_cost, fires_fought, out_of_region = get_route_stats(route_ixs, arcs, costs)
    
    ## store this information to the route_data ##
    
    # add 1 to number of routes for this crew, store the index
    route_data.routes_per_crew[crew] += 1
    ix = route_data.routes_per_crew[crew]
    
    # append the route cost
    route_data.route_costs[crew, ix] = route_cost
    
    # append the fires fought
    route_data.fires_fought[crew, ix, :, :] = fires_fought
    
    # append the out-of-region assignments
    route_data.out_of_reg[crew, ix, :] = out_of_region
    
    return 1

end

update_available_routes (generic function with 1 method)

In [18]:
function get_supp_plan_stats(var_p, var_d, beta, tolerance=0.0001)
    
    # get the cost based on the perimeter progression
    cost = beta * (sum(value.(var_p)) - value(var_p[1])/2 - value(var_p[NUM_TIME_PERIODS+1]/2))
    
    # get the number of crews present each time period from line constructed
    crew_vector = value.(var_d)
    int_crew_vector = convert.(Int64, round.(crew_vector))
    @assert maximum(abs.(crew_vector - int_crew_vector)) < tolerance "Not an integer plan"
    
    return cost, int_crew_vector

end

get_supp_plan_stats (generic function with 2 methods)

In [19]:
function initialize_supp_plan_data(max_supp_plans)
    
    return SuppressionPlanData(zeros(NUM_FIRES), 
                               Matrix{Float64}(undef, NUM_FIRES, max_supp_plans),
                               zeros(Int8, (NUM_FIRES, max_supp_plans, NUM_TIME_PERIODS))
                              )
end

initialize_supp_plan_data (generic function with 1 method)

In [20]:
function update_available_supp_plans(fire, cost, allotment, plan_data)
    
    # add 1 to number of plans for this fire, store the index
    plan_data.plans_per_fire[fire] += 1
    ix = plan_data.plans_per_fire[fire]
    
    # append the route cost
    plan_data.plan_costs[fire, ix] = cost
    
    # append the fires fought
    plan_data.crews_present[fire, ix, :] = allotment
    
    return 1

end

update_available_supp_plans (generic function with 1 method)

In [21]:
function full_formulation(integer_routes, region_data, constraint_data, rotation_order, 
                          costs, progs, perims, beta, gamma, verbose=false)
    
    # get number of arcs
    n_arcs = length(costs)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    regs = region_data.crew_regions
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    
    if ~verbose
        set_optimizer_attribute(m, "OutputFlag", 0)
    end

    # fire suppression plan section
    @variable(m, p[g=1:G, t=1:T+1] >= 0)
    @variable(m, l[g=1:G, t=1:T])
    @variable(m, d[g=1:G, t=1:T] >= 0)
    @constraint(m, perim_growth[g=1:G, t=1:T], p[g, t+1] >= progs[g, t] * 
                                                           (p[g, t] - l[g, t] / 2) - l[g, t] / 2)
    @constraint(m, perim_start[g=1:G], p[g, 1] == perims[g])

    # routing plan section
    if integer_routes
        @variable(m, z[1:n_arcs] >= 0, Int)
    else
        @variable(m, z[1:n_arcs] >= 0)
    end
    
    @variable(m, q[1:C, 0:T] >= 0, Int)
    
    # build out_of_region constraints
    @constraint(m, out_of_region[c=1:C, t=0:T],
    
        # out of region penalty is at least
        q[c, t] >=
        
            # this crew's cumulative rotations
            sum(z[i] for t_0=0:t, i in constraint_data.out_of_region[c, t_0+1]) 
        
        - 
        
            # average cumulative rotations among all crews in same region
            sum(z[i] for c_0 in keys(rotation_order[regs[c]]), t_0=0:t, 
                i in constraint_data.out_of_region[c_0, t_0+1]) /
            length(keys(rotation_order[regs[c]]))
        
        -
        
            # normalizing factor for specific crew rotation order
            (1 - rotation_order[regs[c]][c] / length(keys(rotation_order[regs[c]])))
        
        -
            # big-M for if crew goes not leave region at this time
            T * (1 - sum(z[i] for i in constraint_data.out_of_region[c, t+1]))
        
    )


    @constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=1:2],

            sum(z[constraint_data.f_out[c, g, t, rest]]) ==
            sum(z[constraint_data.f_in[c, g, t, rest]])
    
    )
    
    @constraint(m, base_flow[c=1:C, t=1:T, rest=1:2],

            sum(z[constraint_data.b_out[c, t, rest]]) ==
            sum(z[constraint_data.b_in[c, t, rest]])
    
    )


    @constraint(m, linking[g=1:G, t=1:T],

        sum(z[constraint_data.supp_fire[g, t]]) >= d[g, t] 
    )
    
    @constraint(m, line_building[g=1:G, t=1:T], l[g, t] <= LINE_PER_CREW * d[g, t])

    # build start constraint
    @constraint(m, start[c=1:C], 

        sum(z[constraint_data.start[c]]) == 1
    )
    
    
    

    @objective(m, Min, 
        beta * (sum(p) - sum(p[1:G, 1])/2 - sum(p[1:G, T+1])/2) + 
        sum(z .* costs) + sum(q) * gamma
    )
    
    return m, p, d, z, q, out_of_region
    
end

full_formulation (generic function with 2 methods)

In [22]:
function load_data(path)
    
    # get distance from fire f to fire g 
    fire_dists =  readdlm(path * "/fire_distances.csv", ',')

    # get distance from base c to fire g (NUM_CREWS-by-NUM_FIRES)
    base_fire_dists =  readdlm(path * "/base_fire_distances.csv", ',')

    # initialize travel times (number of periods) from fire f to fire g
    tau = convert(Array{Int}, ones(size(fire_dists)))

    # initialize number of periods to travel from base c to fire g (NUM_CREWS-by-NUM_FIRES)
    tau_base_to_fire = convert(Array{Int}, ones((size(base_fire_dists))))

    # read intial crew statuses (location, period by which they must rest)
    # (-1 in current_fire means crew is currently at base)
    # (rested_periods is the amount of time crew has been at base, relevant for completing rest)
    crew_starts = CSV.read(path * "/sample_crew_starts.csv", DataFrame)
    rest_by = crew_starts[!, "rest_by"]
    current_fire = crew_starts[!, "current_fire"]
    rested_periods = crew_starts[!, "rested_periods"]


    return (GlobalData(fire_dists, base_fire_dists, tau, tau_base_to_fire), 
            CrewStatus(rest_by, current_fire, rested_periods))
end

load_data (generic function with 1 method)

In [23]:
function master_problem(route_data, supp_plan_data, region_data, rotation_order, gamma, price_branch=false)
    
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)
    
    regs = region_data.crew_regions
    
    # decision variables
    if price_branch
        @variable(m, route[c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c]] >= 0, Int)
        @variable(m, plan[g=1:NUM_FIRES, p=1:supp_plan_data.plans_per_fire[g]] >= 0, Int)
        @variable(m, q[c=1:NUM_CREWS, t=0:NUM_TIME_PERIODS] >= 0, Int)
    else
        @variable(m, route[c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c]] >= 0)
        @variable(m, plan[g=1:NUM_FIRES, p=1:supp_plan_data.plans_per_fire[g]] >= 0)
        @variable(m, q[c=1:NUM_CREWS, t=0:NUM_TIME_PERIODS] >= 0)
    end
    
    # constraints that you must choose a plan per crew and per fire
    @constraint(m, route_per_crew[c=1:NUM_CREWS], 
                sum(route[c, r] for r=1:route_data.routes_per_crew[c]) == 1)
    @constraint(m, plan_per_fire[g=1:NUM_FIRES], 
                sum(plan[g, p] for p=1:supp_plan_data.plans_per_fire[g]) >= 1)
    
    # linking constraint
    @constraint(m, linking[g=1:NUM_FIRES, t=1:NUM_TIME_PERIODS],
                    
                    # crews at fire
                    sum(route[c, r] * route_data.fires_fought[c, r, g, t] 
                        for c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c]) 
        
                    >=
        
                    # crews suppressing
                    sum(plan[g, p] * supp_plan_data.crews_present[g, p, t] 
                        for p=1:supp_plan_data.plans_per_fire[g]) 
        
                )
    
    # out_of_region constraint
    @constraint(m, out_of_region[c=1:NUM_CREWS, t=0:NUM_TIME_PERIODS],
    
        # out of region penalty is at least
        q[c, t] >=
        
            # this crew's cumulative rotations
            sum(route[c, r] * route_data.out_of_reg[c, r, t_0 + 1] 
            for r=1:route_data.routes_per_crew[c], t_0=0:t)
        
        - 
        
            # average cumulative rotations among all crews in same region
            sum(route[c_0, r] * route_data.out_of_reg[c_0, r, t_0 + 1] 
                for c_0 in keys(rotation_order[regs[c]]), r=1:route_data.routes_per_crew[c_0],
                t_0=0:t) /
            length(keys(rotation_order[regs[c]]))
        
        -
        
            # normalizing factor for specific crew rotation order
            (1 - rotation_order[regs[c]][c] / length(keys(rotation_order[regs[c]])))
        
        -
            # big-M for if crew goes not leave region at this time
            NUM_TIME_PERIODS * (1 - sum(route[c, r] * route_data.out_of_reg[c, r, t+1]
                                        for r=1:route_data.routes_per_crew[c])
                               )
        
    )
    
    @objective(m, Min, 
        
                  # route costs
                  sum(route[c, r] * route_data.route_costs[c, r] 
                        for c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c])
        
                  +
                     
                  # suppression plan costs
                  sum(plan[g, p] * supp_plan_data.plan_costs[g, p] 
                     for g=1:NUM_FIRES, p=1:supp_plan_data.plans_per_fire[g]) 
        
                  +
        
                  # rotational queueing violations cost
                  sum(q) * gamma
               )
    
    return Dict("m" => m, "q" => q, "sigma" => route_per_crew, "pi" => plan_per_fire, 
                "rho" => linking, "eta" => out_of_region, "route" => route, "plan" => plan)
end 

master_problem (generic function with 2 methods)

In [24]:
function init_route_subproblem(crew_ixs, crew, constraint_data, integer_routes=false)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)

    # routing plan section
    if integer_routes
        @variable(m, z[crew_ixs] >= 0, Int)
    else
        @variable(m, z[crew_ixs] >= 0)
    end


    @constraint(m, fire_flow[g=1:G, t=1:T, rest=1:2],

            sum(z[constraint_data.f_out[crew, g, t, rest]]) ==
            sum(z[constraint_data.f_in[crew, g, t, rest]])
    
    )
    
    @constraint(m, base_flow[t=1:T, rest=1:2],

            sum(z[constraint_data.b_out[crew, t, rest]]) ==
            sum(z[constraint_data.b_in[crew, t, rest]])
    
    )

    # build start constraint
    @constraint(m, start, 

        sum(z[constraint_data.start[crew]]) == 1
    )
    
    return Dict("m" => m, "z" => z, "ff" => fire_flow)
    
end

init_route_subproblem (generic function with 2 methods)

In [25]:
function init_suppression_plan_subproblem(progs, perims, fire, beta)
    
    T = NUM_TIME_PERIODS
    
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)

    # fire suppression plan section
    @variable(m, p[t=1:T+1] >= 0)
    @variable(m, l[t=1:T] >= 0)
    @variable(m, NUM_CREWS >= d[t=1:T] >= 0, Int)
    @constraint(m, suppression_per_crew[t=1:T], l[t] <= d[t] * LINE_PER_CREW)
    @constraint(m, perim_growth[t=1:T], p[t+1] >= progs[fire, t] * (p[t] - l[t] / 2) - l[t] / 2)
    @constraint(m, perim_start, p[1] == perims[fire])
    
#    
    
    return Dict("m" => m, "p" => p, "d" => d, "beta" => beta)
end

init_suppression_plan_subproblem (generic function with 1 method)

In [351]:
function init_suppression_plan_subproblem(config)
             
    if config["solver_type"] == "dp"

        states = create_discrete_fire_states(config["model_data"])
        graphs = generate_graphs(states, config["model_data"], ["ceiling", "floor"])
        nodes_avail = Dict()
        nodes_avail["floor"] = Dict(i => [j for j in 1:size(graphs["floor"])[1] 
                                          if isassigned(graphs["floor"], j, i)] 
                                    for i in 1:size(graphs["floor"])[2])
        nodes_avail["ceiling"] = Dict(i => [j for j in 1:size(graphs["ceiling"])[1] 
                                            if isassigned(graphs["ceiling"], j, i)] 
                                      for i in 1:size(graphs["ceiling"])[2])
        state_costs = zeros(length(states), NUM_TIME_PERIODS + 1)
        for i = 1:length(states)
            for j = 1:NUM_TIME_PERIODS + 1
                state_costs[i, j] = get_state_entrance_cost(states[i], j, config["model_data"])
            end
        end
        
        return Dict("graphs" => graphs, "avail_nodes" => nodes_avail, "state_costs" => state_costs)
        
    elseif config["solver_type"] == "network_flow_gurobi"
        
        # get graph formulation
        d = copy(config)
        d["solver_type"] = "dp"
        sp_dp = init_suppression_plan_subproblem(d)
        
        output = Dict{String, Any}()
        
        for strategy in keys(sp_dp["graphs"])
            
            graph = sp_dp["graphs"][strategy]

            arc_array = arcs_from_state_graph(graph)
            n_arcs = length(arc_array[:, 1])

            # add crew to front
            arc_array = hcat(convert.(Int, zeros(length(arc_array[:, 1]))) .+ fire, arc_array)

            state_costs = sp_dp["state_costs"]
            arc_costs = [state_costs[arc_array[i, 5], arc_array[i, 4]] for i in 1:n_arcs]


            n_states = size(state_costs)[1]  

            out_arcs = Array{Vector{Int64}}(undef, n_states, NUM_TIME_PERIODS)
            in_arcs = Array{Vector{Int64}}(undef, n_states, NUM_TIME_PERIODS)

            for s in 1:n_states
                state_arcs = [i for i in 1:n_arcs if arc_array[i, 2] == s]
                for t in 1:NUM_TIME_PERIODS
                    out_arcs[s, t] = [i for i in state_arcs if arc_array[i, 3] == t]
                end
            end

            for s in 1:n_states
                state_arcs = [i for i in 1:n_arcs if arc_array[i, 5] == s]
                for t in 1:NUM_TIME_PERIODS
                    in_arcs[s, t] = [i for i in state_arcs if arc_array[i, 4] == t]
                end
            end

            # intialize model
            m = Model(() -> Gurobi.Optimizer(GRB_ENV))
            set_optimizer_attribute(m, "OutputFlag", 0)

            @variable(m, z[1:n_arcs] >= 0, Int)
            @constraint(m, flow[s=1:n_states, t=1:NUM_TIME_PERIODS], sum(z[out_arcs[s, t]]) == sum(z[in_arcs[s, t]]))
            @constraint(m, start, z[1] == 1)


            output[strategy] = Dict("arcs" => arc_array, "costs" => arc_costs, "m" => m, "z" => z)
        end
        
        return output
        
    elseif config["solver_type"] == "gurobi"
        
        if config["model_data"]["model_type"] == "simple_linear"

            progs = config["model_data"]["progressions"]
            perim = config["model_data"]["start_perim"]
            beta = config["model_data"]["beta"]

            T = NUM_TIME_PERIODS

            m = Model(() -> Gurobi.Optimizer(GRB_ENV))
            set_optimizer_attribute(m, "OutputFlag", 0)

            # fire suppression plan section
            @variable(m, p[t=1:T+1] >= 0)
            @variable(m, l[t=1:T] >= 0)
            @variable(m, NUM_CREWS >= d[t=1:T] >= 0, Int)
            @constraint(m, suppression_per_crew[t=1:T], l[t] <= d[t] * LINE_PER_CREW)
            @constraint(m, perim_growth[t=1:T], p[t+1] >= progs[t] * (p[t] - l[t] / 2) - l[t] / 2)
            @constraint(m, perim_start, p[1] == perim)


            return Dict("m" => m, "p" => p, "d" => d, "beta" => beta)
        else
            error("Model type not implemented for Gurobi")
        end
    else
        error("Solver type not implemented")
    end
end

init_suppression_plan_subproblem (generic function with 2 methods)

In [372]:
function run_fire_subproblem(sp, config, rho)
    
    if config["solver_type"] == "gurobi"
        
        if config["model_data"]["model_type"] == "simple_linear"
            
            m = sp["m"]
            p = sp["p"]
            d = sp["d"]
            beta = sp["beta"]
            
            if config["warm_start"] == "dummy"
                @objective(m, Min, sum(d))
            elseif config["warm_start"] == false
                @objective(m, Min, beta * (sum(p) - p[1]/2 - p[NUM_TIME_PERIODS + 1]/2) + 
                                   sum(d .* rho) + 0.0001 * sum(d))
            else
                error("warm start type not implemented")
            end
            
            optimize!(m)
            
            # get the required information from the model decision variables
            cost, crew_vector = get_supp_plan_stats(p, d, beta)
            
            return cost, objective_value(m), crew_vector
        else
            error("model type not implemented")
        end
        
    elseif config["solver_type"] == "dp"
        strategy = config["solver_strategy"]
        graph = sp["graphs"][strategy]
        avail_nodes = sp["avail_nodes"][strategy]
        state_enter_costs = sp["state_costs"]
        
        if config["warm_start"] == "dummy"
            duals = zeros(length(rho)) .+ 1e12
        elseif config["warm_start"] == false
            duals = rho
        else
            error("warm start type not implemented")
        end
            
        allotments, states, cost, rel_cost = solve_fire_dp(graph, avail_nodes, duals, state_enter_costs)
        return cost, rel_cost, allotments
        
    elseif config["solver_type"] == "network_flow_gurobi"
        strategy = config["solver_strategy"]
        model_data = sp[strategy]
        m = model_data["m"]
        z = model_data["z"]
        arc_costs = model_data["costs"]
        arc_array = model_data["arcs"]
        
        if config["warm_start"] == "dummy"
            duals = zeros(length(rho)) .+ 1e12
        elseif config["warm_start"] == false
            duals = rho
        else
            error("warm start type not implemented")
        end
        
        # no cost to start edge
        duals = vcat([0], duals)
        
        adj_costs = arc_costs .+ [arc_array[i, 6] * duals[arc_array[i, 3] + 1] for i in 1:length(arc_costs)]
        @objective(m, Min, sum(adj_costs .* z))
        optimize!(m)
        
        rel_cost = objective_value(m)
        arcs_used = [i for i in 1:length(adj_costs) if value(z[i]) > 0.9]
        cost = sum(arc_costs[arcs_used])
        allotments = arc_array[arcs_used[2:length(arcs_used)], 6]

        return cost, rel_cost, allotments
    else
        error("solver type not implemented")
    end
end

run_fire_subproblem (generic function with 1 method)

In [112]:
function initialize_column_generation(arcs, costs, constraint_data, fire_model_configs, solver_configs, max_plans)
    
    # initialize subproblems
    route_sps = []
    for crew in 1:NUM_CREWS
        ixs = [i for i in 1:length(arcs[:, 1]) if arcs[i, 1] == crew]
        d = init_route_subproblem(ixs, crew, constraint_data)
        d["arc_ixs"] = ixs
        push!(route_sps, d)
    end
    
    plan_sps = []
    for fire in 1:NUM_FIRES
        config = solver_configs[fire]
        config["model_data"] = fire_model_configs[fire]
        d = init_suppression_plan_subproblem(config)
        push!(plan_sps, d)
    end
    
    # initialize routes and suppression plans to populate
    routes = initialize_route_data(max_plans)
    suppression_plans = initialize_supp_plan_data(max_plans)
    
    ## generate dummy plans (no suppression) to ensure feasibility at first step ##
    
    # for each crew
    for crew in 1:NUM_CREWS
        
        # get the crew's subproblem instance
        crew_sp = route_sps[crew]
        m = crew_sp["m"]
        z = crew_sp["z"]
        crew_ixs = crew_sp["arc_ixs"]

        # set objective in light of dual variables
        @objective(m, Min, sum(z[ix] * (costs[ix]) for ix in crew_ixs))

        # optimize
        optimize!(m)

        # update crew routes
        crew_arcs = [i for i in crew_ixs if (value(z[i]) > 0.5)]
        update_available_routes(crew, crew_arcs, arcs, costs, routes)
    
    end
    
    # for each fire
    for fire in 1:NUM_FIRES
        
        sp_config = solver_configs[fire]
        sp_config["model_data"] = fire_model_configs[fire]
        sp_config["solver_strategy"] = "ceiling"
        sp_config["warm_start"] = "dummy"
        
        cost, rel_cost, allotment = run_fire_subproblem(plan_sps[fire], sp_config, zeros(NUM_TIME_PERIODS))

        # update suppression plans
        update_available_supp_plans(fire, cost, allotment, suppression_plans)

    end
    
    return ColumnGeneration(route_sps, plan_sps, routes, suppression_plans)
    
end

initialize_column_generation (generic function with 2 methods)

In [113]:
function run_crew_subproblem(sps, crew, costs, local_costs)
    
    # get the crew's subproblem instance
    crew_sp = sps[crew]
    m = crew_sp["m"]
    z = crew_sp["z"]
    crew_ixs = crew_sp["arc_ixs"]
    
    # set objective in light of dual variables
    @objective(m, Min, sum(z[ix] * (local_costs[ix] + costs[ix]) for ix in crew_ixs))
        
    # optimize
    optimize!(m)
    
    return objective_value(m), z
end

run_crew_subproblem (generic function with 1 method)

In [114]:
function run_CG_step(cg, arcs, costs, global_data, region_data, fire_model_configs, solver_configs, rot_order, gamma)
    
    # formulate and solve the master problem
    mp = master_problem(cg.routes, cg.suppression_plans, region_data, rot_order, gamma)
    optimize!(mp["m"])

    # grab the dual variables
    sigma = dual.(mp["sigma"])
    rho = dual.(mp["rho"])
    eta = dual.(mp["eta"])
    pie = dual.(mp["pi"]) # lol can't overwrite "pi" in Julia

    # using the dual variables, get the local adjustments to the arc costs in the route subproblems
    d = Dict("out_of_region_dual" => eta, "region_data"=> region_data, "rotation_order" => rot_order, "linking_dual" => rho)
    local_costs = get_arc_costs(global_data, arcs, d)

    ## run subproblems ##

    # for each fire
    for fire in 1:NUM_FIRES

        # run the subproblem
        sp_config = solver_configs[fire]
        sp_config["model_data"] = fire_model_configs[fire]
        sp_config["solver_strategy"] = "ceiling"
        sp_config["warm_start"] = false
        
        cost, rel_cost, allotment = run_fire_subproblem(cg.plan_sps[fire], sp_config, rho[fire, :])

        # if there is an improving plan
        if rel_cost < pie[fire]

            # add it
            update_available_supp_plans(fire, cost, allotment, cg.suppression_plans)

        end
    end

    # for each crew
    for crew in 1:NUM_CREWS

        # run the crew subproblem
        obj, assignments = run_crew_subproblem(cg.route_sps, crew, costs, local_costs)

        # if there is an improving route
        if obj < sigma[crew]

            # add it
            crew_arcs = [i for i in cg.route_sps[crew]["arc_ixs"] if (value(assignments[i]) > 0.5)]
            update_available_routes(crew, crew_arcs, arcs, costs, cg.routes)

        end

    end 
    return mp
end

run_CG_step (generic function with 2 methods)

In [359]:
in_path = "data/processed"

# get inital fire perimeters and no-suppression progression parameters
M = readdlm(in_path * "/sample_growth_patterns.csv", ',')
start_perims = M[:, 1]
progressions = M[:, 2:15]

NUM_TIME_PERIODS = size(M)[2] - 1 
NUM_FIRES = size(M)[1]      


g_data, crew_status = load_data(in_path)
r_data = RegionData([1, 1, 1, 1, 1, 2, 2, 2, 2, 2], [1, 1, 2, 2])
rotation_order = get_rotation_orders(r_data.crew_regions)
A = generate_arcs(g_data, r_data, crew_status);

rest_pen = get_rest_penalties(crew_status.rest_by, 99999, positive)
cost_params = Dict("cost_per_mile"=> 1, "rest_violation" => rest_pen, "fight_fire" => ALPHA)
arc_costs = get_arc_costs(g_data, A, cost_params)

c_data = define_network_constraint_data(A);

In [360]:
AGG_PREC = 0
PASSIVE_STATES = 0

0

In [368]:
col_gen_data.suppression_plans.crews_present[:, 1, :]

3×14 Matrix{Int8}:
 10  10  10  10  10  10  10  5  0  0  0  0  0  0
 10  10  10   1   0   0   0  0  0  0  0  0  0  0
 10   0   0   0   0   0   0  0  0  0  0  0  0  0

In [405]:
gamma = 0

fire_configs = []
for fire in 1:NUM_FIRES
    model_config = Dict("model_type" => "simple_linear", "progressions" => progressions[fire, :], 
                        "start_perim" => start_perims[fire], "line_per_crew" => LINE_PER_CREW, 
                        "beta" => BETA)
    push!(fire_configs, model_config)
end 


v_s = [0]
e_s = [0]

agg_pass = [(20, 10), (15, 30), (10, 30), (15, 50), (5, 30), (10, 50), (2, 30), (5, 100), (1, 100)] 
agg_pass = [(30, 5, "dp"), (30, 5, "network_flow_gurobi"),
            (20, 10, "dp"), (20, 10, "network_flow_gurobi"),
            (10, 30, "dp"), (10, 30, "network_flow_gurobi"),
            (5, 50, "dp"), (5, 50, "network_flow_gurobi"),
            (2, 50, "dp"), (2, 50, "network_flow_gurobi"),
            (1, 50, "dp"), (1, 50, "network_flow_gurobi")]

df = DataFrame(fire_sp_type=[], total_V=[], total_E=[], iters=[], cg_time=[], lr_pct_gap=[], pb_pct_gap=[])
for param in agg_pass
    
    AGG_PREC = param[1]
    PASSIVE_STATES = param[2]
    
    fire_solver_configs = [Dict{String,Any}("solver_type" => param[3]) for fire in 1:NUM_FIRES]

    max_plans = 1000
    col_gen_data = initialize_column_generation(A, arc_costs, c_data, fire_configs, fire_solver_configs, max_plans)
    
    if param[3] == "dp"
        v_s = [size(col_gen_data.plan_sps[i]["graphs"]["ceiling"])[1] for i in 1:NUM_FIRES]

        e_s = [sum([length(col_gen_data.plan_sps[k]["graphs"]["ceiling"][i, j]) 
                  for i in 1:size(col_gen_data.plan_sps[k]["graphs"]["ceiling"])[1],
                      j in 1:size(col_gen_data.plan_sps[k]["graphs"]["ceiling"])[2]
                      if isassigned(col_gen_data.plan_sps[k]["graphs"]["ceiling"], i, j)
                    ]
                   )
               for k = 1:NUM_FIRES]
    end
    

    
    current_num_routes = copy(col_gen_data.routes.routes_per_crew)
    current_num_plans = copy(col_gen_data.suppression_plans.plans_per_fire)

    objs = []
    max_iters = 500
    n_iters = 0
    opt = false
    tot_time = 0
    max_time = 60

    while (~opt) & (n_iters < max_iters) & (tot_time < max_time)

        n_iters += 1

        tot_time += @elapsed mp = run_CG_step(col_gen_data, A, arc_costs, g_data, r_data, fire_configs, 
                                              fire_solver_configs, rotation_order, gamma)
        push!(objs, objective_value(mp["m"]))

        next_num_routes = col_gen_data.routes.routes_per_crew
        next_num_plans = col_gen_data.suppression_plans.plans_per_fire

        if (sum(next_num_routes) == sum(current_num_routes)) & (sum(next_num_plans) == sum(current_num_plans))
            opt = true
        end

        current_num_routes = copy(next_num_routes)
        current_num_plans = copy(next_num_plans)

    end
    m, p, d, z, q, oor = full_formulation(true, r_data, c_data, rotation_order, arc_costs, 
                              progressions, start_perims, BETA, gamma)
    optimize!(m)
    
    m2, p, d, z, q, oor = full_formulation(false, r_data, c_data, rotation_order, arc_costs, 
                              progressions, start_perims, BETA, 0)
    optimize!(m2)
    
    pb = master_problem(col_gen_data.routes, col_gen_data.suppression_plans, r_data, rotation_order, gamma, true)
    optimize!(pb["m"])
    # pct_gap_30 = 100 * (objs[30] / objs[length(objs)] - 1)

    pb_true_gap = 100 * (objective_value(pb["m"]) / objective_value(m) - 1)
    cg_lr_gap = 100 * (objs[length(objs)] / objective_value(m2) - 1)
    
    push!(df, [fire_solver_configs[1]["solver_type"], sum(v_s), sum(e_s), n_iters, tot_time, cg_lr_gap, pb_true_gap])
end

df

,fire_sp_type,total_V,total_E,iters,cg_time,lr_pct_gap,pb_pct_gap
,Any,Any,Any,Any,Any,Any,Any
1,dp,128,3204,281,60.2283,7.74451,99.5372
2,network_flow_gurobi,128,3204,179,50.4579,7.74451,149.87
3,dp,196,6409,247,60.2077,5.11385,32.937
4,network_flow_gurobi,196,6409,228,60.048,5.11385,48.9245
5,dp,416,15270,249,60.1025,1.573,19.2337
6,network_flow_gurobi,416,15270,95,60.1058,1.573,40.7886
7,dp,796,29594,315,60.128,1.16805,26.8253
8,network_flow_gurobi,796,29594,51,60.0293,1.19939,26.8686
9,dp,1756,60841,218,60.1114,0.913589,26.0066


In [ ]:
	fire_sp_type	total_V	total_E	iters	time	lr_pct_gap	pb_pct_gap
Any	Any	Any	Any	Any	Any	Any
1	dp	0	0	15	0.819473	11.811	117.648


In [ ]:
fire_sp_type	total_V	total_E	iters	time	lr_pct_gap	pb_pct_gap
Any	Any	Any	Any	Any	Any	Any
1	network_flow_gurobi	0	0	30	11.989	2.0177	61.2015


In [393]:
col_gen_data.suppression_plans.crews_present[:, 11, :]

3×14 Matrix{Int8}:
 10  10   0  10  10  10  0  10  6  0  0  0  0  0
  0   0  10  10  10   2  0   0  0  0  0  0  0  0
  7  10   0   0   0   0  0   0  0  0  0  0  0  0

In [394]:
col_gen_data.suppression_plans.crews_present[:, 12, :]

3×14 Matrix{Int8}:
 0  10   0  10  10  10  10  0  10  2  0  0  0  0
 1   0  10   0   0  10  10  0   0  0  0  0  0  0
 9   0   7   2   0   0   0  0   0  0  0  0  0  0

In [395]:
col_gen_data.suppression_plans.crews_present[:, 13, :]

3×14 Matrix{Int8}:
 0  0  10  10  0  10  10  10  0  0  6  0  0  0
 1  9   0   9  0   0  10   2  0  0  0  0  0  0
 9  2   0   0  0   0   0   0  0  0  0  0  0  0

In [396]:
col_gen_data.suppression_plans.crews_present[:, 14, :]

3×14 Matrix{Int8}:
 0  9  9  10  0  0  10  0  10  7  0  0  0  0
 0  9  0   0  0  9   9  3   0  0  0  0  0  0
 9  0  8   0  0  0   0  0   0  0  0  0  0  0

In [397]:
col_gen_data.suppression_plans.crews_present[:, 15, :]

3×14 Matrix{Int8}:
 0  0  0  0  1  10   0  10  10  10  0  0  0  0
 0  0  0  9  0   0  10  10   2   0  0  0  0  0
 9  0  7  2  0   0   0   0   0   0  0  0  0  0

In [387]:
col_gen_data.suppression_plans.crews_present[:, 11, :]

3×14 Matrix{Int8}:
 10  10   0  10  10  10  0  10  6  0  0  0  0  0
  0   0  10  10  10   2  0   0  0  0  0  0  0  0
  7  10   0   0   0   0  0   0  0  0  0  0  0  0

In [388]:
col_gen_data.suppression_plans.crews_present[:, 12, :]

3×14 Matrix{Int8}:
 0  10   0  10  10  10  10  0  10  2  0  0  0  0
 1   0  10   0   0  10  10  0   0  0  0  0  0  0
 9   0   7   2   0   0   0  0   0  0  0  0  0  0

In [389]:
col_gen_data.suppression_plans.crews_present[:, 13, :]

3×14 Matrix{Int8}:
 0  0  10  10  0  10  10  10  0  0  6  0  0  0
 1  9   0   9  0   0  10   2  0  0  0  0  0  0
 9  2   0   0  0   0   0   0  0  0  0  0  0  0

In [390]:
col_gen_data.suppression_plans.crews_present[:, 14, :]

3×14 Matrix{Int8}:
 0  9  9  10  0  0  10  0  10  7  0  0  0  0
 0  9  0   0  0  9   9  3   0  0  0  0  0  0
 9  0  8   0  0  0   0  0   0  0  0  0  0  0

In [391]:
col_gen_data.suppression_plans.crews_present[:, 15, :]

3×14 Matrix{Int8}:
 0  0  0  0  1  10   0  10  10  10  0  0  0  0
 0  0  0  9  0   0  10  10   2   0  0  0  0  0
 9  0  7  2  0   0   0   0   0   0  0  0  0  0

In [ ]:
	fire_sp_type	total_V	total_E	iters	time	lr_pct_gap	pb_pct_gap
Any	Any	Any	Any	Any	Any	Any
1	network_flow_gurobi	0	0	50	23.2881	1.59401	40.7886


In [ ]:
0.0 100.0 24.26995010000001 72.0 0.0746409992447461 76.95710933220496 0.7957642612639093

In [ ]:
0.0 67.0 8.5660698 61.0 0.1137069451359718 46.46200837797869 0.573370980375687

In [322]:
function arcs_from_state_graph(graph)
    
    visitable = [(i,j) for i in 1:size(graph)[1], j in 1:size(graph)[2] if isassigned(graph, i, j)]
    edges = []

    for (i, j) in visitable
        push!(edges, copy(reduce(hcat, [[i, j, j+1, a[1], a[2]] for a in graph[i, j]])'))
    end

    return vcat([size(graph)[1], 0, 1, size(graph)[1], 0]', reduce(vcat, edges))
end

arcs_from_state_graph (generic function with 1 method)

In [341]:
fire = 1

sp_dp = col_gen_data.plan_sps[fire]
ceil_graph = sp_dp["graphs"]["ceiling"]

arc_array = arcs_from_state_graph(ceil_graph)
n_arcs = length(arc_array[:, 1])

# add crew to front
arc_array = hcat(convert.(Int, zeros(length(arc_array[:, 1]))) .+ fire, arc_array)

state_costs = sp_dp["state_costs"]
arc_costs = [state_costs[arc_array[i, 5], arc_array[i, 4]] for i in 1:n_arcs]

  
n_states = size(state_costs)[1]  

out_arcs = Array{Vector{Int64}}(undef, n_states, NUM_TIME_PERIODS)
in_arcs = Array{Vector{Int64}}(undef, n_states, NUM_TIME_PERIODS)

for s in 1:n_states
    state_arcs = [i for i in 1:n_arcs if arc_array[i, 2] == s]
    for t in 1:NUM_TIME_PERIODS
        out_arcs[s, t] = [i for i in state_arcs if arc_array[i, 3] == t]
    end
end

for s in 1:n_states
    state_arcs = [i for i in 1:n_arcs if arc_array[i, 5] == s]
    for t in 1:NUM_TIME_PERIODS
        in_arcs[s, t] = [i for i in state_arcs if arc_array[i, 4] == t]
    end
end

# intialize model
m = Model(() -> Gurobi.Optimizer(GRB_ENV))
# set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, z[1:n_arcs] >= 0, Int)
@constraint(m, flow[s=1:n_states, t=1:NUM_TIME_PERIODS], sum(z[out_arcs[s, t]]) == sum(z[in_arcs[s, t]]))
@constraint(m, start, z[1] == 1)


Dict("arcs" => arc_array, "costs" => arc_costs, "m" => m, "z" => z)

Dict{String, Any} with 4 entries:
  "m"     => A JuMP Model…
  "arcs"  => [1 232 … 232 0; 1 232 … 130 10; … ; 1 13 … 6 1; 1 13 … 7 0]
  "costs" => [50000.0, 1.29648e5, 1.31658e5, 1.33668e5, 1.35678e5, 1.37688e5, 1…
  "z"     => VariableRef[z[1], z[2], z[3], z[4], z[5], z[6], z[7], z[8], z[9], …

In [342]:
@objective(m, Min, sum(arc_costs .* z));
optimize!(m)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3249 rows, 6407 columns and 12765 nonzeros
Model fingerprint: 0xa615e234
Variable types: 0 continuous, 6407 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+02, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 670100.50251
Presolve removed 3249 rows and 6407 columns
Presolve time: 0.04s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 549497 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.494974874372e+05, best bound 5.494974874372e+05, gap 0.0000%

User-callback calls 266, time in user-callback 0.00 sec


In [343]:
arcs_used = [i for i in 1:n_arcs if value(z[i]) > 0.99]
arc_costs[arcs_used]

15-element Vector{Float64}:
  50000.0
 129648.24120603016
 113567.8391959799
  97487.43718592965
  72361.80904522612
  49246.23115577889
  29145.72864321608
   8040.2010050251265
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0

In [344]:
objective_value(m)

549497.487437186

In [328]:
@time 

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 7029 rows, 12378 columns and 24661 nonzeros
Model fingerprint: 0x89800b5a
Variable types: 0 continuous, 12378 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 669047.61905
Presolve removed 6372 rows and 5909 columns
Presolve time: 3.14s
Presolved: 657 rows, 6469 columns, 12703 nonzeros
Found heuristic solution: objective 635964.91228
Variable types: 0 continuous, 6469 integer (1062 binary)

Root relaxation: objective 5.367168e+05, 237 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    536716.79198 536716.792  0.00%     -    3s

Explored 0 nodes (237 simplex iterations) in 3.16 seconds
Thread 

In [226]:
in_arcs[8, 14]

1-element Vector{Int64}:
 2638

In [ ]:
# intialize model
m = Model(() -> Gurobi.Optimizer(GRB_ENV))
set_optimizer_attribute(m, "OutputFlag", 0)

# routing plan section
if integer_routes
    @variable(m, z[crew_ixs] >= 0, Int)
else
    @variable(m, z[crew_ixs] >= 0)
end


@constraint(m, fire_flow[g=1:G, t=1:T, rest=1:2],

        sum(z[constraint_data.f_out[crew, g, t, rest]]) ==
        sum(z[constraint_data.f_in[crew, g, t, rest]])

)

@constraint(m, base_flow[t=1:T, rest=1:2],

        sum(z[constraint_data.b_out[crew, t, rest]]) ==
        sum(z[constraint_data.b_in[crew, t, rest]])

)

# build start constraint
@constraint(m, start, 

    sum(z[constraint_data.start[crew]]) == 1
)

return Dict("m" => m, "z" => z, "ff" => fire_flow)

In [193]:
fire_arcs = []

for fire in 1:NUM_FIRES
    
    sp_dp = col_gen_data.plan_sps[fire]
    ceil_graph = sp_dp["graphs"]["ceiling"]

    arc_array = arcs_from_state_graph(ceil_graph)

    # add crew to front
    arc_array = hcat(convert.(Int, zeros(length(arc_array[:, 1]))) .+ fire, arc_array)
    
    push!(fire_arcs, arc_array)
    
end

reduce(vcat, fire_arcs)

6409×5 Matrix{Int64}:
 1  112   1  65  10
 1  112   1  66   9
 1  112   1  67   8
 1  112   1  68   7
 1  112   1  69   6
 1  112   1  70   5
 1  112   1  72   4
 1  112   1  73   3
 1  112   1  74   2
 1  112   1  75   1
 1  112   1  76   0
 1   65   2  57  10
 1   65   2  58   9
 ⋮               
 3   11  14  10   9
 3   11  14  11   0
 3   12  14  12   0
 3   13  14  13   0
 3   14  14  14   0
 3   15  14  15   0
 3   16  14  16   0
 3   17  14  17   0
 3   18  14  18   0
 3   19  14  19   0
 3   20  14  20   0
 3   21  14  21   0

In [44]:
function update_fire_stats(curr_stats, curr_time, crew_allocation, params)
    
    if params["model_type"] == "simple_linear"
        
        line_per_crew = params["line_per_crew"]
        prog = params["progressions"][curr_time]
        line = line_per_crew * crew_allocation
        next_stats = (curr_stats - line/2) * prog - line/2
        next_stats = max(0, next_stats)
        
    else
        error("Not implemented")
    end
    
    return next_stats
end

update_fire_stats (generic function with 1 method)

In [45]:
function inverse_update_fire_stats(stats_from, stats_to, time_from, time_to, params)
    """ Returns number of crews needed for given fire transition """
    
    if params["model_type"] == "simple_linear"

        line_per_crew = params["line_per_crew"]
        prog = params["progressions"][time_from]
        
        crews = 2 / line_per_crew * (prog * stats_from - stats_to) / (1 + prog)
        
    else
        error("Not implemented")
    end
    
    return crews
    
end

inverse_update_fire_stats (generic function with 1 method)

In [134]:
function create_discrete_fire_states(params)
    
    if params["model_type"] == "simple_linear"
        
        # get the no-suppression progression of this fire
        progs = params["progressions"]
        start_perim = params["start_perim"]
        no_supp = [start_perim]
        for i in 1:NUM_TIME_PERIODS
            push!(no_supp, no_supp[i] * progs[i])
        end
        
        # generalize this later
        aggressive_precision = AGG_PREC
        num_aggressive_states = convert(Int, round(start_perim * 2 / aggressive_precision))
        num_passive_states = PASSIVE_STATES

        aggressive_states = LinRange(0, num_aggressive_states * aggressive_precision, num_aggressive_states)
        passive_states = exp.(LinRange(log(num_aggressive_states * aggressive_precision + 1), 
                                       maximum(log.(no_supp .+ 1)), num_passive_states + 1)
                             )
        passive_states = passive_states[2:num_passive_states+1] .- 1
        all_states = vcat(aggressive_states, passive_states)
        all_states = vcat(all_states, 9999999)
        
        push!(all_states, start_perim)
        
    else
        error("Not implemented")
    end
    
    return all_states
end

create_discrete_fire_states (generic function with 1 method)

In [47]:
function generate_state_transition_crew_reqs(curr_stats, curr_time, sorted_states, params, round_types)
    
    if params["model_type"] == "simple_linear"
        
        # get all possibly feasible states
        min_state_val = update_fire_stats(curr_stats, curr_time, NUM_CREWS + 0.5, params)
        max_state_val = update_fire_stats(curr_stats, curr_time, 0, params)
        
        # get min and max index of the possibly feasible states
        min_state_ix = searchsorted(sorted_states, min_state_val).start
        max_state_ix = searchsorted(sorted_states, max_state_val).start
        
        # inititalize output 
        output = Dict()
        for round_type in round_types
            output[round_type] = []
        end
        
        # inititalize minimum crews needed so far, for trimming (explained below)
        min_used = Dict()
        for round_type in round_types
            min_used[round_type] = NUM_CREWS + 1
        end
        
        # for each feasible state
        for state_ix in min_state_ix:max_state_ix
            crews_needed = inverse_update_fire_stats(curr_stats, sorted_states[state_ix], curr_time, 
                                                     curr_time + 1, params)
            
            # for each round type
            for round_type in round_types
                
                # round the number of crews
                if round_type == "ceiling"
                    crews = max(0, convert(Int, ceil(crews_needed - 0.0001)))
                elseif round_type == "floor"
                    crews = max(0, convert(Int, floor(crews_needed + 0.0001)))
                elseif round_type == "nearest"
                    crews = max(0, convert(Int, round(crews_needed)))
                else
                    error("Round type not implemented")
                end
                
                # since the states are sorted in increasing level of cost and risk
                # we can trim arcs that are dominated
                
                # if this is a feasible number of crews and we do not have a dominating arc
                if (crews <= NUM_CREWS) & (crews < min_used[round_type])
                    
                    # update the minimum crews used for this round type
                    min_used[round_type] = crews
                    
                    # push the crew requirement for this transition to the corresponding list
                    push!(output[round_type], (state_ix, crews))
                end
            end
        end
        
        return output

    
    else
        error("Model type not implemented")
    end
    
end
        

generate_state_transition_crew_reqs (generic function with 1 method)

In [48]:
function generate_graphs(states, params, round_types)
    
    # separate out non start states
    non_start_states = states[1:length(states)-1]
    
    # initializations
    n_states = length(non_start_states)
    crews_needed = Dict()
    
    # for each round type
    for round_type in round_types
        
        # init graph as array of vectors (each index is a state*time vertex, each vector holds edges out)
        crews_needed[round_type] = Array{Vector{}}(undef, n_states + 1, NUM_TIME_PERIODS + 1)
        
        # initialize time, state indices to check next
        curr_time = 1
        next_to_check = [n_states + 1]
        
        # while we have not yet reached the end of the horizon
        while curr_time < NUM_TIME_PERIODS + 1
            
            # copy the indices to check and reset next_to_check
            to_check = copy(next_to_check)
            next_to_check = []

            # for each state index feasible to reach at this time period
            for check in to_check

                # if it is not the last (no return) state
                if check != n_states
                    
                    # find the crew requirements for feasible edges
                    edges = generate_state_transition_crew_reqs(states[check], curr_time, non_start_states, 
                                                                params, [round_type])[round_type]
                
                # otherwise stay at this state
                else
                    edges = [(n_states, 0)]
                end

                # update the crews needed for each edge coming out of this state
                crews_needed[round_type][check, curr_time] = edges

                # append the neighbors to next_to_check
                next_to_check = vcat(next_to_check, [edges[i][1] for i in 1:length(edges) 
                                                     if ~(edges[i][1] in next_to_check)])
            end
            
            # add one to the time
            curr_time += 1
        end
    end
    
    return crews_needed
end

generate_graphs (generic function with 1 method)

In [49]:
function solve_fire_dp(graph, avail_nodes, rho, state_enter_costs)
    
    n_states = size(state_enter_costs)[1]
    
    curr_time = NUM_TIME_PERIODS
    costs = zeros(n_states, NUM_TIME_PERIODS + 1) .+ 1.0e12
    costs[:, NUM_TIME_PERIODS + 1] = state_enter_costs[:, NUM_TIME_PERIODS + 1]
    bests = Dict()
    
    # backward induction
    while curr_time > 0
    
        state_time_costs = state_enter_costs[:, curr_time]
        nodes_to_check = avail_nodes[curr_time]
        
        for node in nodes_to_check
            current_cost = 1e10
            current_best = -1
            current_allot = -1
            edges = graph[node, curr_time]
            for edge in edges
                edge_cost = costs[edge[1], curr_time + 1] + rho[curr_time] * edge[2]
                if edge_cost < current_cost
                    current_best = edge[1]
                    current_allot = edge[2]
                    current_cost = edge_cost
                end
            end
            costs[node, curr_time] = current_cost + state_time_costs[node]
            bests[(node, curr_time)] = (current_best, current_allot);
        end

        curr_time -= 1
    end
    
    # recreate path
    curr_state = n_states
    curr_time = 1
    curr_cost = state_enter_costs[curr_state, curr_time]
    allotments = convert.(Int, zeros(NUM_TIME_PERIODS))
    all_states = convert.(Int, zeros(NUM_TIME_PERIODS + 1))
    all_states[1] = curr_state

    while curr_time < NUM_TIME_PERIODS + 1

        next_state = bests[(curr_state, curr_time)]
        curr_state = next_state[1]
        allotments[curr_time] = next_state[2]
        curr_time += 1
        all_states[curr_time] = curr_state
        curr_cost += state_enter_costs[curr_state, curr_time]
    end
    
    return allotments, all_states, curr_cost, costs[n_states, 1]
end

solve_fire_dp (generic function with 1 method)

In [50]:
function get_state_entrance_cost(state, enter_time, params)
    
    if params["model_type"] == "simple_linear"
        
        if (enter_time == 1) | (enter_time == NUM_TIME_PERIODS + 1)
            return params["beta"] * state / 2
        elseif (enter_time > 1) & (enter_time < NUM_TIME_PERIODS + 1)
            return params["beta"] * state
        else
            error("Invalid time")
        end
    else
        error("Not implemented")
    end
end

get_state_entrance_cost (generic function with 1 method)

In [561]:
function get_cost(fire_arc, states, dual_linking)
    
    cost = 0
    if (fire_arc[3] == 1) | (fire_arc[3] == NUM_TIME_PERIODS) 
        cost = states[fire_arc[4]] / 2
    else
        cost = states[fire_arc[4]]
    end
    
    cost += fire_arc[5] * dual_linking[fire_arc[1], fire_arc[3]]
    
    return cost
end

get_cost (generic function with 2 methods)

In [311]:
function get_alphas(state, prog, sorted_states)
    
    min_state_val = new_perim(state, prog, NUM_CREWS, LINE_PER_CREW)
    max_state_val = new_perim(state, prog, 0, LINE_PER_CREW)
    min_state_ix = searchsorted(sorted_states, min_state_val).start
    max_state_ix = searchsorted(sorted_states, max_state_val).start
    
    edges = []
    for state_ix in min_state_ix:max_state_ix
        push!(edges, (state_ix, get_crews_needed_for_transition(state, 
                                                                 sorted_states[state_ix], 
                                                                 prog,
                                                                 LINE_PER_CREW,
                                                                 "ceiling")
                       ))
    end
    
    return edges

end

get_alphas (generic function with 1 method)

In [562]:
function generate_fire_arcs(g, progs, perims)
    
    # get the progressions for this specific fire
    fire_progs = progs[g, :]
    
    no_supp = [perims[g]]
    for i in 1:NUM_TIME_PERIODS
        push!(no_supp, no_supp[i] * progs[i])
    end

    aggressive_precision = 15
    num_aggressive_states = convert(Int, round(perims[g] * 2 / aggressive_precision))
    num_passive_states = 30

    aggressive_states = LinRange(0, num_aggressive_states * aggressive_precision, num_aggressive_states)
    passive_states = exp.(LinRange(log(num_aggressive_states * aggressive_precision+ 1), maximum(log.(no_supp .+ 1)), num_passive_states + 1))
    passive_states = passive_states[2:num_passive_states+1] .- 1
    all_states = vcat(aggressive_states, passive_states)
    all_states = vcat(all_states, 9999999)

    # generate arcs
    states_appended = copy(all_states)
    push!(states_appended, start_perims[g])
    s = length(all_states)
    crews_needed = Array{Vector}(undef, s + 1, NUM_TIME_PERIODS + 1)
    curr_time = 1
    state_name = 0
    next_to_check = [s + 1]

    while curr_time < NUM_TIME_PERIODS + 1

        to_check = copy(next_to_check)
        next_to_check = []

        for check in to_check
            curr_state = states_appended[check]

            if check != s
                edges = get_alphas(curr_state, fire_progs[curr_time], all_states)
            else
                edges = [(s, 0)]
            end

            for edge in edges
                crews_needed[check, curr_time] = edges
            end
            next_to_check = vcat(next_to_check, [edges[i][1] for i in 1:length(edges) if ~(edges[i][1] in next_to_check)])
        end
        curr_time += 1
    end
    
    visitable = [(i,j) for i in 1:size(crews_needed)[1], j in 1:size(crews_needed)[2] if isassigned(crews_needed, i, j)]
    
    edges = []

    for (i, j) in visitable
        push!(edges, copy(reduce(hcat, [[i, j, a[1], a[2]] for a in crews_needed[i, j]])'))
    end
    
    arc_array = reduce(vcat, edges)
    
    # add crew to front
    arc_array = hcat(convert.(Int, zeros(length(arc_array[:, 1]))) .+ g, arc_array)
    
    return crews_needed, arc_array, states_appended
end

generate_fire_arcs (generic function with 1 method)

In [59]:
function new_perim(old_perim, prog, num_crews, line_per_crew)
    
    line = line_per_crew * num_crews
    return (old_perim - line/2) * prog - line/2

end

new_perim (generic function with 1 method)

In [194]:
function get_crews_needed_for_transition(state_1, state_2, prog, line_per_crew, round_type)
    
    crews = 2 / line_per_crew * (prog * state_1 - state_2) / (1 + prog)
    
    if round_type == "nearest"
        crews = convert(Int, round(crews))
    
    elseif round_type == "ceiling"
        crews = convert(Int, ceil(crews - 0.0001))
    end
    
    return max(crews, 0) 
    
end 

get_crews_needed_for_transition (generic function with 2 methods)

In [563]:
graph, fire_arc_arr, fire_states = generate_fire_arcs(2, progressions, start_perims);

In [9]:
function get_out_of_region_stats(region, arcs_used, region_data)
    """
    """
    
    # restrict to the arcs that exited the given region
    out_of_region_ixs = [i for i in length(arcs_used[:, 1]) if arcs_used[i, 10] == region]
    out_of_region_arcs = arcs_used[out_of_region_ixs, :]
    
    # get the crews associated with this region
    crews = [i for i in 1:length(region_data.crew_regions) if region_data.crew_regions[i] == region]
    
    # initialize output array of indicator variables for crews exiting region
    out_array = zeros(length(crews), NUM_TIME_PERIODS)
    
    # for each crew in the region
    for i in 1:length(crews)
        
        # restrict to the arcs involving this crew
        crew_ixs = [j for j in length(out_of_region_arcs[:, 1]) if arcs_used[j, 1] == crews[i]]
        crew_arcs = out_of_region_arcs[crew_ixs, :]
        
        # get the times of rotation
        rotation_times = [t in crew_arcs[:, 6] ? 1 : 0 for t in 1:NUM_TIME_PERIODS]
        
        # update output for crew
        out_array[i, :] = rotation_times
        
    end
    
    return out_array
end

LoadError: syntax: incomplete: "function" at In[9]:1 requires end